# Anomaly investigation in heat pump

In [2]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import stumpy

import DataRetriever as dr

retriever = dr.DataRetriever()
hour = retriever.get_data("All-Subsystems-hour-Year2.pkl")

In [7]:
heatpump_df = hour[['HVAC_HeatPumpIndoorUnitPower', 'HVAC_HeatPumpOutdoorUnitPower']] / 1e3 #to kWh
heatpump_df

,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower
Timestamp,,
2015-02-01 00:00:00,0.175449,1.045818
2015-02-01 01:00:00,0.391606,1.042027
2015-02-01 02:00:00,0.388485,1.023145
2015-02-01 03:00:00,0.175793,1.066177
2015-02-01 04:00:00,0.371121,1.081113
...,...,...
2016-01-31 19:00:00,0.011230,0.022489
2016-01-31 20:00:00,0.011367,0.022483
2016-01-31 21:00:00,0.011449,0.022510


In [ ]:
# The heat pump unit was not working for approximately two days starting on May 30, 2015 due to the failure of a relay on the outdoor unit.

In [8]:
m = 24*2
mps, indices = stumpy.mstump(heatpump_df, m = m)
mps

array([[4.91365783, 4.9230866 , 4.89544469, ..., 2.89835076, 2.9788805 ,
        2.99666051],
       [6.139823  , 6.17455934, 6.13497431, ..., 3.36103523, 3.41933613,
        3.46081253]])

In [13]:
mps[1]

array([6.139823  , 6.17455934, 6.13497431, ..., 3.36103523, 3.41933613,
       3.46081253])

In [9]:
indices

array([[ 576,  577,  578, ..., 4713, 4714, 4738],
       [ 240,  241,  242, ..., 1341, 1342, 1343]], dtype=int64)

In [11]:
motifs_idx = np.argmin(mps, axis=1)
motifs_idx

array([7493, 5814], dtype=int64)

In [12]:
nn_idx = indices[np.arange(len(motifs_idx)), motifs_idx]
nn_idx

array([7400, 7400], dtype=int64)

In [15]:
fig = make_subplots(rows=4,
                    subplot_titles=(" ", "", "", "")
                    )

fig.add_trace(
    go.Scatter(x=heatpump_df.index, y=heatpump_df["HVAC_HeatPumpOutdoorUnitPower"], name="Outdoor Unit"),
    row=1, col=1)

fig.add_trace(
    go.Scatter(x=heatpump_df.index, y=heatpump_df["HVAC_HeatPumpIndoorUnitPower"], name="Indoor Unit"),
    row=2, col=1)

fig.add_trace(
    go.Scatter(y = mps[0], name="First MP"),
    row=3, col=1)

fig.add_trace(
    go.Scatter(y = mps[1], name="Second MP"),
    row=4, col=1)

fig.show()

## Heat pump 1 dim

In [16]:
heat_mp = stumpy.stump(heatpump_df["HVAC_HeatPumpOutdoorUnitPower"], m)
heat_mp

array([[4.913657834328052, 576, -1, 576],
       [4.923086597540987, 577, -1, 577],
       [4.895444690456593, 578, -1, 578],
       ...,
       [2.8983507622434126, 4713, 4713, -1],
       [2.97888049837934, 4714, 4714, -1],
       [2.996660507555145, 4738, 4738, -1]], dtype=object)

In [17]:
fig = make_subplots(rows=2, cols=1
                    )

fig.add_trace(
    go.Scatter(x=heatpump_df.index, y=heatpump_df["HVAC_HeatPumpOutdoorUnitPower"], name="Outdoor Unit"),
    row=1, col=1)

fig.add_trace(
    go.Scatter(y = heat_mp[:, 0], name="MP"),
    row=2, col=1)

fig.show()